# Training Multilabel Classification Models with Legal NLP


![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings_JSL/Legal/04.2.Training_Legal_Multiclass_Classifier.ipynb)

In this notebook, you will learn how to use Spark NLP and Legal NLP to train multilabel classification models.

Let`s dive in!

# Colab Setup

In [ ]:
# Install the johnsnowlabs library to access Spark-OCR and Spark-NLP for Healthcare, Finance, and Legal.
! pip install -q johnsnowlabs

In [ ]:
from johnsnowlabs import nlp, legal
# After uploading your license run this to install all licensed Python Wheels and pre-download Jars the Spark Session JVM
# Make sure to restart your notebook afterwards for changes to take effect
nlp.install(force_browser=True)

## Start Spark Session

In [3]:
from johnsnowlabs import nlp, legal 
# Automatically load license data and start a session with all jars user has access to
spark = nlp.start()

👌 Launched cpu optimized session with with: 🚀Spark-NLP==4.2.4, 💊Spark-Healthcare==4.2.4, running on ⚡ PySpark==3.1.2


## Introduction

For the text classification tasks, we will use two annotators:

- `MultiClassifierDL`: `Multilabel Classification` (can predict more than one class for each text) using a Bidirectional GRU with Convolution architecture built with TensorFlow that supports up to 100 classes. The inputs are Sentence Embeddings such as state-of-the-art UniversalSentenceEncoder, BertSentenceEmbeddings or SentenceEmbeddings.
- `ClassifierDL`: uses the state-of-the-art Universal Sentence Encoder as an input for text classifications. Then, a deep learning model (DNNs) built with TensorFlow that supports `Binary Classification` and `Multiclass Classification` (up to 100 classes).

The `ClassifierDLApproach` annotator trains a multiclass model, where the predictions is one category out of a predifined set of categories that are present in the training data.

## Loading the data

In [4]:
! wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings_JSL/Finance/data/finance_clf_data.csv

In [5]:
import pandas as pd
df = pd.read_csv('finance_clf_data.csv', encoding="utf8")
print(f"Shape of the full dataset: {df.shape}")

Shape of the full dataset: (6128, 3)


In [6]:
df.head()

,text,label,len
0,Presently we do not believe any U S or State r...,business,402
1,\nnetwork outages or performance degradation ...,risk_factors,496
2,Available Information\nOur reports filed with ...,business,356
3,\n 42 530\n \n \n \n \n \n 42 530\nTotal liab...,financial_statements,359
4,8\nTable of Contents\ndevelopment employee eng...,business,582


In [7]:
df['label'].value_counts()

risk_factors               1926
financial_statements       1888
business                    970
financial_conditions        346
form_10k_summary            240
executives_compensation     155
controls_procedures         138
equity                      111
market_risk                 100
executives                   73
legal_proceedings            51
properties                   48
security_ownership           46
exhibits                     36
Name: label, dtype: int64

Since the deep learning models can take some time to train, we will limit our dataset to a smaller number of observations in order to illustrate how to use Spark NLP and Finance NLP annotators and pipelines to train the model, but without having to wait too much.

Please note that the quality and the quantity of training data is very relevant to the obtained trianed model, and the results we obtain here are for illustration purposes only. To obtain a more realistic model, pelase consider using the full dataset or addin extra observations from different sources. 

In [8]:
from sklearn.model_selection import train_test_split

# The top 3 categories (number of observations)
filter_classes = ["risk_factors", "financial_statements", "business"]

# We make a random sample with 1000 observations
df = df.loc[df.label.isin(filter_classes)].sample(1000)

# Stratify split for train and test datasets
train_data, test_data = train_test_split(
    df, train_size=0.9, stratify=df.label, random_state=42
)

# Send to spark
train = spark.createDataFrame(train_data)
test = spark.createDataFrame(test_data)

In [9]:
from pyspark.sql.functions import col

train.groupBy("label").count().orderBy(col("count").desc()).show()

+--------------------+-----+
|               label|count|
+--------------------+-----+
|        risk_factors|  373|
|financial_statements|  343|
|            business|  184|
+--------------------+-----+



In [10]:
from pyspark.sql.functions import col

test.groupBy("label").count().orderBy(col("count").desc()).show()

+--------------------+-----+
|               label|count|
+--------------------+-----+
|        risk_factors|   41|
|financial_statements|   38|
|            business|   21|
+--------------------+-----+



 ## With Universal Encoder

In [11]:
document_assembler = (
    nlp.DocumentAssembler().setInputCol("text").setOutputCol("document")
)

embeddings = (
    nlp.UniversalSentenceEncoder.pretrained()
    .setInputCols("document")
    .setOutputCol("sentence_embeddings")
)

classsifierdl = (
    legal.ClassifierDLApproach()
    .setInputCols(["sentence_embeddings"])
    .setOutputCol("class")
    .setLabelColumn("label")
    .setMaxEpochs(30)
    .setEnableOutputLogs(True)
    .setOutputLogsPath("multiclass_use_logs")
    .setLr(0.001)
    .setBatchSize(4)
    .setDropout(0.15)
)

clf_pipeline = nlp.Pipeline(stages=[document_assembler, embeddings, classsifierdl])

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[OK!]


In [12]:
%%time
clf_pipelineModel = clf_pipeline.fit(train)

CPU times: user 623 ms, sys: 69.2 ms, total: 692 ms
Wall time: 1min 33s


In [13]:
import os
log_file_name = os.listdir("multiclass_use_logs")[0]

with open("multiclass_use_logs/"+log_file_name, "r") as log_file :
    print(log_file.read())

Training started - epochs: 30 - learning_rate: 0.001 - batch_size: 4 - training_examples: 900 - classes: 3
Epoch 0/30 - 2.89s - loss: 187.18683 - acc: 0.7188889 - batches: 225
Epoch 1/30 - 2.40s - loss: 153.52151 - acc: 0.8666667 - batches: 225
Epoch 2/30 - 2.19s - loss: 147.76152 - acc: 0.8888889 - batches: 225
Epoch 3/30 - 3.05s - loss: 145.0456 - acc: 0.8977778 - batches: 225
Epoch 4/30 - 3.31s - loss: 143.36841 - acc: 0.9033333 - batches: 225
Epoch 5/30 - 5.03s - loss: 142.06673 - acc: 0.9066667 - batches: 225
Epoch 6/30 - 4.27s - loss: 140.97423 - acc: 0.91555554 - batches: 225
Epoch 7/30 - 2.26s - loss: 140.05913 - acc: 0.92 - batches: 225
Epoch 8/30 - 2.60s - loss: 139.31575 - acc: 0.9222222 - batches: 225
Epoch 9/30 - 2.51s - loss: 138.73438 - acc: 0.92444444 - batches: 225
Epoch 10/30 - 1.90s - loss: 138.28282 - acc: 0.9266667 - batches: 225
Epoch 11/30 - 1.98s - loss: 137.91661 - acc: 0.93 - batches: 225
Epoch 12/30 - 1.93s - loss: 137.61601 - acc: 0.93333334 - batches: 225
E

In [14]:
preds = clf_pipelineModel.transform(test)

In [15]:
preds_df = preds.select("label", "text", "class.result").toPandas()
preds_df.head()

,label,text,result
0,financial_statements,2020\n2019\n2018\nCurrent tax provision \n \nF...,[financial_statements]
1,financial_statements,required to be independent with respect to the...,[financial_statements]
2,business,\n \nTrading Software \n \no\nOrder Managemen...,[business]
3,business,\nExpand our strategic and regional partnersh...,[business]
4,financial_statements,Variable Consideration \nWe record revenue fro...,[financial_statements]


In [16]:
# The result is an array since in Spark NLP you can have multiple sentences.
# Let's explode the array and get the item(s) inside of result column out
preds_df['result'] = preds_df['result'].apply(lambda x : x[0])

In [17]:
# We are going to use sklearn to evalute the results on test dataset
from sklearn.metrics import classification_report

print(classification_report(preds_df["label"], preds_df["result"]))

                      precision    recall  f1-score   support

            business       0.85      0.81      0.83        21
financial_statements       0.93      1.00      0.96        38
        risk_factors       0.90      0.85      0.88        41

            accuracy                           0.90       100
           macro avg       0.89      0.89      0.89       100
        weighted avg       0.90      0.90      0.90       100



## With RoBerta Embeddings

**Please restart your runtime to get rid of the out-of-memory error and read dataset again**

We do not have Legal Sentence Embeddings yet, But we can use the Legal RoBerta Embeddings and then average them.

In [1]:
from johnsnowlabs import nlp, legal 
# Automatically load license data and start a session with all jars user has access to
spark = nlp.start()

📋 Loading license number 0 from /root/.johnsnowlabs/licenses/license_number_0_for_Spark-Healthcare_Spark-OCR.json
👌 Launched cpu optimized session with with: 🚀Spark-NLP==4.2.4, 💊Spark-Healthcare==4.2.4, running on ⚡ PySpark==3.1.2


In [2]:
import pandas as pd
df = pd.read_csv('finance_clf_data.csv', encoding="utf8")
print(f"Shape of the full dataset: {df.shape}")

Shape of the full dataset: (6128, 3)


In [3]:
from sklearn.model_selection import train_test_split

# The top 3 categories (number of observations)
filter_classes = ["risk_factors", "financial_statements", "business"]

# We make a random sample with 1000 observations
df = df.loc[df.label.isin(filter_classes)].sample(1000)

# Stratify split for train and test datasets
train_data, test_data = train_test_split(
    df, train_size=0.9, stratify=df.label, random_state=42
)

# Send to spark
train = spark.createDataFrame(train_data)
test = spark.createDataFrame(test_data)

In [4]:
from pyspark.sql.functions import col

train.groupBy("label").count().orderBy(col("count").desc()).show()

+--------------------+-----+
|               label|count|
+--------------------+-----+
|financial_statements|  370|
|        risk_factors|  365|
|            business|  165|
+--------------------+-----+



In [5]:
from pyspark.sql.functions import col

test.groupBy("label").count().orderBy(col("count").desc()).show()

+--------------------+-----+
|               label|count|
+--------------------+-----+
|financial_statements|   41|
|        risk_factors|   41|
|            business|   18|
+--------------------+-----+



In [6]:
embeddings = (
    nlp.RoBertaEmbeddings.pretrained("roberta_embeddings_legal_roberta_base", "en")
    .setInputCols(["document", "token"])
    .setOutputCol("embeddings")
    .setMaxSentenceLength(512)
)

roberta_embeddings_legal_roberta_base download started this may take some time.
Approximate size to download 447.2 MB
[OK!]


In [7]:
document_assembler = (
    nlp.DocumentAssembler().setInputCol("text").setOutputCol("document")
)

tokenizer = nlp.Tokenizer().setInputCols(["document"]).setOutputCol("token")

embeddingsSentence = (
    nlp.SentenceEmbeddings()
    .setInputCols(["document", "embeddings"])
    .setOutputCol("sentence_embeddings")
    .setPoolingStrategy("AVERAGE")
)

classifierdl = (
    legal.ClassifierDLApproach()
    .setInputCols(["sentence_embeddings"])
    .setOutputCol("class")
    .setLabelColumn("label")
    .setMaxEpochs(5)
    .setLr(0.001)
    .setEnableOutputLogs(True)
    .setOutputLogsPath("multiclass_roberta_logs")
    .setBatchSize(4)
    .setDropout(0.15)
)

clf_pipeline = nlp.Pipeline(
    stages=[document_assembler, tokenizer, embeddings, embeddingsSentence, classifierdl]
)

In [8]:
%%time
clf_pipelineModel = clf_pipeline.fit(train)

CPU times: user 12.6 s, sys: 1.7 s, total: 14.3 s
Wall time: 31min 35s


In [9]:
preds = clf_pipelineModel.transform(test)

In [10]:
preds_df = preds.select("label", "text", "class.result").toPandas()

In [13]:
import os
log_files = os.listdir("multiclass_roberta_logs")

with open("multiclass_roberta_logs/"+log_files[0], "r") as log_file :
    print(log_file.read())

Training started - epochs: 5 - learning_rate: 0.001 - batch_size: 4 - training_examples: 900 - classes: 3
Epoch 0/5 - 3.64s - loss: 175.66095 - acc: 0.74222225 - batches: 225
Epoch 1/5 - 3.25s - loss: 151.07233 - acc: 0.8755556 - batches: 225
Epoch 2/5 - 3.33s - loss: 141.50848 - acc: 0.9077778 - batches: 225
Epoch 3/5 - 3.27s - loss: 138.98373 - acc: 0.91888887 - batches: 225
Epoch 4/5 - 3.24s - loss: 137.17067 - acc: 0.9266667 - batches: 225



In [14]:
preds_df.head()

,label,text,result
0,financial_statements,All of our revenue is derived from external cu...,[financial_statements]
1,financial_statements,above market returns to investors \n \n \nPre...,[financial_statements]
2,business,Intellectual Property\nOur success depends in ...,[risk_factors]
3,financial_statements,767 \n\nLess Future production tax expense \n...,[financial_statements]
4,financial_statements,3 141 \n \nBalance end of period\n \n3 389 \n ...,[business]


In [15]:
# Let's explode the array and get the item(s) inside of result column out
preds_df['result'] = preds_df['result'].apply(lambda x : x[0])

from sklearn.metrics import classification_report

print (classification_report(preds_df['label'], preds_df['result']))


                      precision    recall  f1-score   support

            business       0.87      0.72      0.79        18
financial_statements       0.93      0.95      0.94        41
        risk_factors       0.91      0.95      0.93        41

            accuracy                           0.91       100
           macro avg       0.90      0.87      0.89       100
        weighted avg       0.91      0.91      0.91       100



## Save model and Zip it for Modelshub Upload/Downloads

In [16]:
# Save a Spark NLP model
clf_pipelineModel.stages[-1].write().overwrite().save('ClfBert')

# cd into saved dir and zip
! cd /content/ClfBert ; zip -r /content/ClfBert.zip *

  adding: classifierdl_tensorflow (deflated 58%)
  adding: fields/ (stored 0%)
  adding: fields/datasetParams/ (stored 0%)
  adding: fields/datasetParams/_SUCCESS (stored 0%)
  adding: fields/datasetParams/.part-00000.crc (stored 0%)
  adding: fields/datasetParams/._SUCCESS.crc (stored 0%)
  adding: fields/datasetParams/.part-00001.crc (stored 0%)
  adding: fields/datasetParams/part-00001 (deflated 30%)
  adding: fields/datasetParams/part-00000 (deflated 26%)
  adding: metadata/ (stored 0%)
  adding: metadata/_SUCCESS (stored 0%)
  adding: metadata/.part-00000.crc (stored 0%)
  adding: metadata/._SUCCESS.crc (stored 0%)
  adding: metadata/part-00000 (deflated 40%)
